<a href="https://colab.research.google.com/github/za4n/A-budgetApp/blob/master/vtryModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install kaggle package
!pip install kaggle

# Create directory for Kaggle API token
!mkdir -p ~/.kaggle

In [2]:
# Move the Kaggle API token to the correct location
!cp kaggle.json ~/.kaggle/
# Set correct permissions
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Create a project directory in your Google Drive
!mkdir -p "/content/drive/My Drive/VITON_HD_Project"

Mounted at /content/drive


In [4]:
# Create necessary subdirectories
!mkdir -p "/content/drive/My Drive/VITON_HD_Project/dataset"
!mkdir -p "/content/drive/My Drive/VITON_HD_Project/models"
!mkdir -p "/content/drive/My Drive/VITON_HD_Project/results"

In [5]:
# Change to the dataset directory
%cd "/content/drive/My Drive/VITON_HD_Project/dataset"

/content/drive/My Drive/VITON_HD_Project/dataset


In [6]:
# Download the dataset
!kaggle datasets download marquis03/high-resolution-viton-zalando-dataset


Dataset URL: https://www.kaggle.com/datasets/marquis03/high-resolution-viton-zalando-dataset
License(s): CC-BY-NC-SA-4.0
100% 4.38G/4.39G [01:03<00:00, 98.3MB/s]
100% 4.39G/4.39G [01:03<00:00, 73.6MB/s]


In [7]:
!unzip -q high-resolution-viton-zalando-dataset.zip

In [8]:
# Check if GPU is being recognized
import torch
print("GPU Available:", torch.cuda.is_available())
print("GPU Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

GPU Available: True
GPU Device Name: Tesla T4


In [9]:
!pip install torch torchvision
!pip install opencv-python-headless
!pip install numpy pandas matplotlib

In [1]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms

# Verify GPU
print("GPU Available:", torch.cuda.is_available())
print("GPU Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

# Check dataset structure
base_path = '/content/drive/MyDrive/VITON_HD_Project/dataset'
print("\nDataset contents:")
print(os.listdir(base_path))

GPU Available: True
GPU Device Name: Tesla T4

Dataset contents:
['high-resolution-viton-zalando-dataset.zip', 'test', 'test_pairs.txt', 'train', 'train_pairs.txt']


In [2]:
# Read and display first few pairs from test_pairs.txt
pairs_file = os.path.join(base_path, 'test_pairs.txt')
print("First 5 lines of test_pairs.txt:")
with open(pairs_file, 'r') as f:
    for i, line in enumerate(f):
        if i < 5:
            print(line.strip())

First 5 lines of test_pairs.txt:
05006_00.jpg 11001_00.jpg
02532_00.jpg 14096_00.jpg
03921_00.jpg 08015_00.jpg
12419_00.jpg 01944_00.jpg
12562_00.jpg 14025_00.jpg


DATA SET CLASS

In [3]:
class VITONDataset(Dataset):
    def __init__(self, root_dir, is_train=True, transform=None):
        self.root_dir = root_dir
        self.is_train = is_train
        self.transform = transform
        self.split = 'train' if is_train else 'test'

        # Get all image files from the directory
        self.image_dir = os.path.join(root_dir, self.split, 'image')
        self.cloth_dir = os.path.join(root_dir, self.split, 'cloth')

        # Get list of files
        self.image_files = sorted(os.listdir(self.image_dir))
        self.cloth_files = sorted(os.listdir(self.cloth_dir))

        # Take only first 100 files for testing
        self.image_files = self.image_files[:100]
        self.cloth_files = self.cloth_files[:100]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Get file names
        image_name = self.image_files[idx]
        cloth_name = self.cloth_files[idx]  # For simplicity, using same index

        # Construct full paths
        image_path = os.path.join(self.image_dir, image_name)
        cloth_path = os.path.join(self.cloth_dir, cloth_name)

        # Load images
        try:
            person_img = Image.open(image_path).convert('RGB')
            cloth_img = Image.open(cloth_path).convert('RGB')
        except Exception as e:
            print(f"Error loading images: {e}")
            print(f"Image path: {image_path}")
            print(f"Cloth path: {cloth_path}")
            raise e

        if self.transform:
            person_img = self.transform(person_img)
            cloth_img = self.transform(cloth_img)

        return {
            'person': person_img,
            'cloth': cloth_img,
            'person_name': image_name,
            'cloth_name': cloth_name
        }

# Define transforms
transform = transforms.Compose([
    transforms.Resize((512, 384)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [4]:
# Initialize dataset with a try-except block
try:
    # Create dataset instance
    dataset_path = '/content/drive/MyDrive/VITON_HD_Project/dataset'
    train_dataset = VITONDataset(dataset_path, is_train=True, transform=transform)

    print(f"Dataset size: {len(train_dataset)}")

    # Create dataloader
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

    # Try loading one batch
    for batch in train_loader:
        print("\nSuccessfully loaded batch:")
        print("Person image shape:", batch['person'].shape)
        print("Cloth image shape:", batch['cloth'].shape)
        print("Person filename:", batch['person_name'])
        print("Cloth filename:", batch['cloth_name'])
        break

except Exception as e:
    print(f"Error occurred: {str(e)}")

Dataset size: 100

Successfully loaded batch:
Person image shape: torch.Size([1, 3, 512, 384])
Cloth image shape: torch.Size([1, 3, 512, 384])
Person filename: ['00054_00.jpg']
Cloth filename: ['00054_00.jpg']


In [5]:
try:
    train_image_dir = os.path.join(base_path, 'train', 'image')
    train_cloth_dir = os.path.join(base_path, 'train', 'cloth')

    print("Files in train/image directory:")
    print(os.listdir(train_image_dir)[:5])  # First 5 files

    print("\nFiles in train/cloth directory:")
    print(os.listdir(train_cloth_dir)[:5])  # First 5 files

except Exception as e:
    print(f"Error checking directories: {str(e)}")

Files in train/image directory:
['00000_00.jpg', '00001_00.jpg', '00002_00.jpg', '00003_00.jpg', '00005_00.jpg']

Files in train/cloth directory:
['00000_00.jpg', '00001_00.jpg', '00002_00.jpg', '00003_00.jpg', '00005_00.jpg']


In [1]:
import torch
import gc
import os
import psutil
import numpy as np
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import matplotlib.pyplot as plt
from tqdm import tqdm

def clear_memory():
    """Clear GPU and RAM memory"""
    # Clear GPU
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

    # Clear RAM
    gc.collect()

    # Delete existing models
    try:
        del model
        del optimizer
        del train_loader
        del test_loader
    except:
        pass

    # Print memory status
    if torch.cuda.is_available():
        print(f"GPU Memory allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB")
        print(f"GPU Memory cached: {torch.cuda.memory_reserved()/1e9:.2f} GB")

    process = psutil.Process(os.getpid())
    print(f"RAM Used: {process.memory_info().rss/1e9:.2f} GB")

# Set environment variable for memory management
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Clear everything
clear_memory()

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set random seeds
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

GPU Memory allocated: 0.00 GB
GPU Memory cached: 0.00 GB
RAM Used: 0.59 GB
Using device: cuda


ENHANCED DATASET

In [2]:
class ImprovedVITONDataset(Dataset):
    def __init__(self, root_dir, is_train=True, max_samples=2000):
        self.root_dir = root_dir
        self.is_train = is_train
        self.split = 'train' if is_train else 'test'

        # Get all directories
        self.image_dir = os.path.join(root_dir, self.split, 'image')
        self.cloth_dir = os.path.join(root_dir, self.split, 'cloth')
        self.segment_dir = os.path.join(root_dir, self.split, 'image-parse')

        # Get files with limit
        self.image_files = sorted(os.listdir(self.image_dir))[:max_samples]
        self.cloth_files = sorted(os.listdir(self.cloth_dir))[:max_samples]

        # Advanced augmentation
        self.transform = transforms.Compose([
            transforms.Resize((512, 384)),
            transforms.RandomHorizontalFlip(p=0.3),
            transforms.ColorJitter(
                brightness=0.2,
                contrast=0.2,
                saturation=0.2,
                hue=0.1
            ),
            transforms.RandomAffine(
                degrees=5,
                translate=(0.05, 0.05),
                scale=(0.95, 1.05)
            ),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        try:
            # Load images
            image_name = self.image_files[idx]
            cloth_name = self.cloth_files[idx]

            image_path = os.path.join(self.image_dir, image_name)
            cloth_path = os.path.join(self.cloth_dir, cloth_name)

            person_img = Image.open(image_path).convert('RGB')
            cloth_img = Image.open(cloth_path).convert('RGB')

            # Transform images
            if self.transform:
                person_img = self.transform(person_img)
                cloth_img = self.transform(cloth_img)

            return {
                'person': person_img,
                'cloth': cloth_img,
                'person_name': image_name,
                'cloth_name': cloth_name
            }

        except Exception as e:
            print(f"Error loading image {image_name}: {str(e)}")
            return self.__getitem__((idx + 1) % self.__len__())

MODEL ARCHITECTURE

In [3]:
class EnhancedVITONModel(nn.Module):
    def __init__(self):
        super().__init__()

        # Feature Extraction
        vgg = models.vgg19(pretrained=True)
        self.feature_extraction = nn.Sequential(*list(vgg.features.children())[:18])
        for param in self.feature_extraction.parameters():
            param.requires_grad = False

        # Attention Module
        self.attention = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.InstanceNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 1, kernel_size=1),
            nn.Sigmoid()
        )

        # Enhanced Warping Module
        self.warping = nn.Sequential(
            nn.Conv2d(512, 256, 3, padding=1),
            nn.InstanceNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, 3, padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 64, 3, padding=1),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 2, 3, padding=1),
            nn.Tanh()
        )

        # Generator Encoder
        self.enc1 = nn.Sequential(
                  nn.Conv2d(6, 64, 3, padding=1),
                  nn.InstanceNorm2d(64),
                  nn.ReLU(inplace=True)
              )
        self.enc2 = nn.Sequential(
                  nn.Conv2d(64, 128, 3, padding=1),
                  nn.InstanceNorm2d(128),
                  nn.ReLU(inplace=True),
                  nn.MaxPool2d(2)
              )
        self.enc3 = nn.Sequential(
                  nn.Conv2d(128, 256, 3, padding=1),
                  nn.InstanceNorm2d(256),
                  nn.ReLU(inplace=True),
                  nn.MaxPool2d(2)
              )
        self.enc4 = nn.Sequential(
                  nn.Conv2d(256, 512, 3, padding=1),
                  nn.InstanceNorm2d(512),
                  nn.ReLU(inplace=True)
              )


        # Generator Decoder with correct dimensions
        self.dec1 = nn.Sequential(
            nn.ConvTranspose2d(512, 256, 4, stride=2, padding=1),
            nn.InstanceNorm2d(256),
            nn.ReLU(inplace=True)
        )

        # Additional upsampling layer for d1 to match e3
        self.upsample_d1 = nn.Upsample(
                size=(128, 96),  # Match e3 size
                mode='bilinear',
                align_corners=True
            )
        self.upsample_d2 = nn.Upsample(
                size=(256, 192),  # Match e2 size
                mode='bilinear',
                align_corners=True
            )
        self.upsample_d3 = nn.Upsample(
                size=(512, 384),  # Match e1 size
                mode='bilinear',
                align_corners=True
            )

        self.dec2 = nn.Sequential(
                nn.Conv2d(512, 256, 3, padding=1),
                nn.InstanceNorm2d(256),
                nn.ReLU(inplace=True),
                nn.Conv2d(256, 128, 3, padding=1)
            )

        self.dec3 = nn.Sequential(
                nn.Conv2d(256, 128, 3, padding=1),
                nn.InstanceNorm2d(128),
                nn.ReLU(inplace=True)
            )

        self.dec4 = nn.Sequential(
                nn.Conv2d(192, 64, 3, padding=1),
                nn.InstanceNorm2d(64),
                nn.ReLU(inplace=True),
                nn.Conv2d(64, 3, 3, padding=1),
                nn.Tanh()
            )



        # Upsampling for flow field
        self.upsample = nn.Upsample(
            size=(512, 384),
            mode='bilinear',
            align_corners=True
        )

    def forward(self, person_img, cloth_img):
        # Extract features
        with torch.no_grad():
            person_features = self.feature_extraction(person_img)
            cloth_features = self.feature_extraction(cloth_img)

        # Apply attention
        attention_mask = self.attention(cloth_features)
        cloth_features = cloth_features * attention_mask

        # Generate flow field
        combined_features = torch.cat([person_features, cloth_features], dim=1)
        flow_field = self.warping(combined_features)
        flow_field = self.upsample(flow_field)

        # Generate sampling grid
        batch_size = person_img.size(0)
        grid = F.affine_grid(
            torch.eye(2, 3).unsqueeze(0).repeat(batch_size, 1, 1).to(person_img.device),
            size=person_img.size(),
            align_corners=True
        )

        # Warp cloth
        warped_cloth = F.grid_sample(
            cloth_img,
            grid + flow_field.permute(0, 2, 3, 1),
            mode='bilinear',
            padding_mode='border',
            align_corners=True
        )
        x = torch.cat([person_img, warped_cloth], dim=1)
        print("\nEncoder shapes:")
        e1 = self.enc1(x)
        print(f"e1: {e1.shape}")

        e2 = self.enc2(e1)
        print(f"e2: {e2.shape}")

        e3 = self.enc3(e2)
        print(f"e3: {e3.shape}")

        e4 = self.enc4(e3)
        print(f"e4: {e4.shape}")

        # Decoder with debugging
        print("\nDecoder shapes:")
        # First decoder block
        d1 = self.dec1(e4)
        print(f"d1 after ConvTranspose: {d1.shape}")

        d1 = self.upsample_d1(d1)
        print(f"d1 after upsample: {d1.shape}")
        print(f"e3 for concatenation: {e3.shape}")

        d1_cat = torch.cat([d1, e3], dim=1)
        print(f"d1_cat: {d1_cat.shape}")

        # Second decoder block
        d2 = self.dec2(d1_cat)
        print(f"d2 before upsample: {d2.shape}")

        d2 = self.upsample_d2(d2)
        print(f"d2 after upsample: {d2.shape}")
        print(f"e2 for concatenation: {e2.shape}")

        d2_cat = torch.cat([d2, e2], dim=1)
        print(f"d2_cat: {d2_cat.shape}")

        # Third decoder block
        d3 = self.dec3(d2_cat)
        print(f"d3 before upsample: {d3.shape}")

        d3 = self.upsample_d3(d3)
        print(f"d3 after upsample: {d3.shape}")
        print(f"e1 for concatenation: {e1.shape}")

        d3_cat = torch.cat([d3, e1], dim=1)
        print(f"d3_cat: {d3_cat.shape}")

        # Final output
        output = self.dec4(d3_cat)
        print(f"\nFinal output: {output.shape}")

        return output


    def get_warped_cloth(self, person_img, cloth_img):
        """Helper function to visualize intermediate results"""
        with torch.no_grad():
            # Extract features
            person_features = self.feature_extraction(person_img)
            cloth_features = self.feature_extraction(cloth_img)

            # Apply attention
            attention_mask = self.attention(cloth_features)
            cloth_features = cloth_features * attention_mask

            # Generate flow field
            combined_features = torch.cat([person_features, cloth_features], dim=1)
            flow_field = self.warping(combined_features)
            flow_field = self.upsample(flow_field)

            # Generate grid and warp
            batch_size = person_img.size(0)
            grid = F.affine_grid(
                torch.eye(2, 3).unsqueeze(0).repeat(batch_size, 1, 1).to(person_img.device),
                size=person_img.size(),
                align_corners=True
            )

            warped_cloth = F.grid_sample(
                cloth_img,
                grid + flow_field.permute(0, 2, 3, 1),
                mode='bilinear',
                padding_mode='border',
                align_corners=True
            )

            return warped_cloth, attention_mask

def test_enhanced_model():
    model = EnhancedVITONModel().to(device)
    person = torch.randn(4, 3, 512, 384).to(device)
    cloth = torch.randn(4, 3, 512, 384).to(device)

    try:
        # Add shape printing for debugging
        output = model(person, cloth)
        warped_cloth, attention = model.get_warped_cloth(person, cloth)

        print("Shapes:")
        print(f"Output: {output.shape}")
        print(f"Warped cloth: {warped_cloth.shape}")
        print(f"Attention mask: {attention.shape}")
        print("\nModel test successful!")

    except Exception as e:
        print(f"Error: {str(e)}")

# Test the model
test_enhanced_model()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Encoder shapes:
e1: torch.Size([4, 64, 512, 384])
e2: torch.Size([4, 128, 256, 192])
e3: torch.Size([4, 256, 128, 96])
e4: torch.Size([4, 512, 128, 96])

Decoder shapes:
d1 after ConvTranspose: torch.Size([4, 256, 256, 192])
d1 after upsample: torch.Size([4, 256, 128, 96])
e3 for concatenation: torch.Size([4, 256, 128, 96])
d1_cat: torch.Size([4, 512, 128, 96])
d2 before upsample: torch.Size([4, 128, 128, 96])
d2 after upsample: torch.Size([4, 128, 256, 192])
e2 for concatenation: torch.Size([4, 128, 256, 192])
d2_cat: torch.Size([4, 256, 256, 192])
d3 before upsample: torch.Size([4, 128, 256, 192])
d3 after upsample: torch.Size([4, 128, 512, 384])
e1 for concatenation: torch.Size([4, 64, 512, 384])
d3_cat: torch.Size([4, 192, 512, 384])

Final output: torch.Size([4, 3, 512, 384])
Shapes:
Output: torch.Size([4, 3, 512, 384])
Warped cloth: torch.Size([4, 3, 512, 384])
Attention mask: torch.Size([4, 1, 128, 96])

Model test successful!


LOSS FUNCTION

In [4]:
class CompositeLoss(nn.Module):
    def __init__(self):
        super().__init__()
        # VGG for perceptual loss
        vgg = models.vgg19(pretrained=True).features[:29].eval()
        for param in vgg.parameters():
            param.requires_grad = False
        self.vgg = vgg.to(device)

        # Loss components
        self.l1_loss = nn.L1Loss()
        self.mse_loss = nn.MSELoss()

    def perceptual_loss(self, x, target):
        """VGG based perceptual loss"""
        vgg_x = self.vgg(x)
        vgg_target = self.vgg(target)
        return F.mse_loss(vgg_x, vgg_target)

    def style_loss(self, x, target):
        """Gram matrix based style loss"""
        def gram_matrix(feat):
            b, ch, h, w = feat.size()
            feat = feat.view(b, ch, h * w)
            gram = torch.bmm(feat, feat.transpose(1, 2))
            return gram.div(ch * h * w)

        x_gram = gram_matrix(self.vgg(x))
        target_gram = gram_matrix(self.vgg(target))
        return F.mse_loss(x_gram, target_gram)

    def forward(self, output, target, warped_cloth=None):
        # Calculate different loss components
        l1 = self.l1_loss(output, target)
        perceptual = self.perceptual_loss(output, target)
        style = self.style_loss(output, target)

        # Warping loss if available
        warp_loss = torch.tensor(0.0).to(device)
        if warped_cloth is not None:
            warp_loss = self.l1_loss(warped_cloth, target)

        # Combine losses with weights
        total_loss = (0.5 * l1 +
                     0.2 * perceptual +
                     0.2 * style +
                     0.1 * warp_loss)

        return total_loss

Training Functions

In [5]:
def train_epoch(model, dataloader, criterion, optimizer, device, gradient_accumulation_steps):
    model.train()
    total_loss = 0
    progress_bar = tqdm(dataloader, desc='Training')

    for batch_idx, batch in enumerate(progress_bar):
        # Load data
        person_images = batch['person'].to(device)
        cloth_images = batch['cloth'].to(device)

        # Clear gradients
        optimizer.zero_grad()

        try:
            # Forward pass
            outputs = model(person_images, cloth_images)

            # Calculate loss
            loss = criterion(outputs, person_images)

            # Normalize loss for gradient accumulation
            loss = loss / gradient_accumulation_steps

            # Backward pass
            loss.backward()

            # Update weights if needed
            if (batch_idx + 1) % gradient_accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()

            # Update metrics
            total_loss += loss.item() * gradient_accumulation_steps
            progress_bar.set_postfix({'loss': loss.item() * gradient_accumulation_steps})

            # Clear memory periodically
            if batch_idx % 10 == 0:
                clear_memory()

        except RuntimeError as e:
            print(f"Error in batch {batch_idx}: {str(e)}")
            continue

    return total_loss / len(dataloader)

def evaluate_model(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    total_psnr = 0

    with torch.no_grad():
        for batch in test_loader:
            person_images = batch['person'].to(device)
            cloth_images = batch['cloth'].to(device)

            try:
                outputs = model(person_images, cloth_images)
                loss = criterion(outputs, person_images)

                # Calculate PSNR
                mse = F.mse_loss(outputs, person_images)
                psnr = 10 * torch.log10(1 / mse)

                total_loss += loss.item()
                total_psnr += psnr.item()

            except RuntimeError as e:
                print(f"Error during evaluation: {str(e)}")
                continue

    return total_loss / len(test_loader), total_psnr / len(test_loader)

TRAINING AND EVALUATION

In [ ]:
# Clear memory before starting
clear_memory()

# Initialize model
model = EnhancedVITONModel().to(device)

# Training parameters
num_epochs = 30
batch_size = 4
gradient_accumulation_steps = 2
learning_rate = 0.0001

# Initialize optimizer and scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5, verbose=True
)

# Initialize loss function
criterion = CompositeLoss().to(device)

# Create dataloaders
dataset_path = "/content/drive/MyDrive/VITON_HD_Project/dataset"
train_dataset = ImprovedVITONDataset(dataset_path, is_train=True, max_samples=2000)
test_dataset = ImprovedVITONDataset(dataset_path, is_train=False, max_samples=200)

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    pin_memory=True
)

# Training metrics tracking
class MetricsTracker:
    def __init__(self):
        self.train_losses = []
        self.test_losses = []
        self.psnr_scores = []

    def update(self, train_loss, test_loss, psnr):
        self.train_losses.append(train_loss)
        self.test_losses.append(test_loss)
        self.psnr_scores.append(psnr)

    def plot_metrics(self):
        plt.figure(figsize=(15, 5))

        plt.subplot(1, 3, 1)
        plt.plot(self.train_losses)
        plt.title('Training Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')

        plt.subplot(1, 3, 2)
        plt.plot(self.test_losses)
        plt.title('Test Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')

        plt.subplot(1, 3, 3)
        plt.plot(self.psnr_scores)
        plt.title('PSNR Score')
        plt.xlabel('Epoch')
        plt.ylabel('PSNR (dB)')

        plt.tight_layout()
        plt.show()

# Initialize metrics tracker
metrics = MetricsTracker()

# Training loop
best_psnr = 0
for epoch in range(num_epochs):
    print(f'\nEpoch [{epoch + 1}/{num_epochs}]')

    # Train
    train_loss = train_epoch(
        model, train_loader, criterion, optimizer, device, gradient_accumulation_steps
    )

    # Evaluate
    test_loss, psnr = evaluate_model(model, test_loader, criterion, device)
    print(f'Train Loss: {train_loss:.4f}')
    print(f'Test Loss: {test_loss:.4f}')
    print(f'PSNR: {psnr:.2f} dB')

    # Update metrics
    metrics.update(train_loss, test_loss, psnr)

    # Update learning rate
    scheduler.step(test_loss)

    # Save best model
    if psnr > best_psnr:
        best_psnr = psnr
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'psnr': psnr,
        }, f'/content/drive/MyDrive/VITON_HD_Project/models/best_model.pth')
        print(f'New best model saved with PSNR: {psnr:.2f} dB')

    # Regular checkpoint
    if (epoch + 1) % 5 == 0:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'psnr': psnr,
        }, f'/content/drive/MyDrive/VITON_HD_Project/models/checkpoint_epoch_{epoch+1}.pth')

    # Clear memory
    clear_memory()

# Plot final metrics
metrics.plot_metrics()
print("Training completed!")

GPU Memory allocated: 0.01 GB
GPU Memory cached: 0.04 GB
RAM Used: 0.90 GB


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(



Epoch [1/30]


Training:   0%|          | 0/500 [00:00<?, ?it/s]


Encoder shapes:
e1: torch.Size([4, 64, 512, 384])
e2: torch.Size([4, 128, 256, 192])
e3: torch.Size([4, 256, 128, 96])
e4: torch.Size([4, 512, 128, 96])

Decoder shapes:
d1 after ConvTranspose: torch.Size([4, 256, 256, 192])
d1 after upsample: torch.Size([4, 256, 128, 96])
e3 for concatenation: torch.Size([4, 256, 128, 96])
d1_cat: torch.Size([4, 512, 128, 96])
d2 before upsample: torch.Size([4, 128, 128, 96])
d2 after upsample: torch.Size([4, 128, 256, 192])
e2 for concatenation: torch.Size([4, 128, 256, 192])
d2_cat: torch.Size([4, 256, 256, 192])
d3 before upsample: torch.Size([4, 128, 256, 192])
d3 after upsample: torch.Size([4, 128, 512, 384])
e1 for concatenation: torch.Size([4, 64, 512, 384])
d3_cat: torch.Size([4, 192, 512, 384])

Final output: torch.Size([4, 3, 512, 384])


...


Encoder shapes:
e1: torch.Size([4, 64, 512, 384])
e2: torch.Size([4, 128, 256, 192])
e3: torch.Size([4, 256, 128, 96])
e4: torch.Size([4, 512, 128, 96])

Decoder shapes:
d1 after ConvTranspose: torch.Size([4, 256, 256, 192])
d1 after upsample: torch.Size([4, 256, 128, 96])
e3 for concatenation: torch.Size([4, 256, 128, 96])
d1_cat: torch.Size([4, 512, 128, 96])
d2 before upsample: torch.Size([4, 128, 128, 96])
d2 after upsample: torch.Size([4, 128, 256, 192])
e2 for concatenation: torch.Size([4, 128, 256, 192])
d2_cat: torch.Size([4, 256, 256, 192])
d3 before upsample: torch.Size([4, 128, 256, 192])
d3 after upsample: torch.Size([4, 128, 512, 384])
e1 for concatenation: torch.Size([4, 64, 512, 384])
d3_cat: torch.Size([4, 192, 512, 384])

Final output: torch.Size([4, 3, 512, 384])


Training:  29%|██▉       | 147/500 [06:26<09:53,  1.68s/it, loss=0.741]


Encoder shapes:
e1: torch.Size([4, 64, 512, 384])
e2: torch.Size([4, 128, 256, 192])
e3: torch.Size([4, 256, 128, 96])
e4: torch.Size([4, 512, 128, 96])

Decoder shapes:
d1 after ConvTranspose: torch.Size([4, 256, 256, 192])
d1 after upsample: torch.Size([4, 256, 128, 96])
e3 for concatenation: torch.Size([4, 256, 128, 96])
d1_cat: torch.Size([4, 512, 128, 96])
d2 before upsample: torch.Size([4, 128, 128, 96])
d2 after upsample: torch.Size([4, 128, 256, 192])
e2 for concatenation: torch.Size([4, 128, 256, 192])
d2_cat: torch.Size([4, 256, 256, 192])
d3 before upsample: torch.Size([4, 128, 256, 192])
d3 after upsample: torch.Size([4, 128, 512, 384])
e1 for concatenation: torch.Size([4, 64, 512, 384])
d3_cat: torch.Size([4, 192, 512, 384])

Final output: torch.Size([4, 3, 512, 384])


Training:  30%|██▉       | 148/500 [06:28<09:50,  1.68s/it, loss=0.581]


Encoder shapes:
e1: torch.Size([4, 64, 512, 384])
e2: torch.Size([4, 128, 256, 192])
e3: torch.Size([4, 256, 128, 96])
e4: torch.Size([4, 512, 128, 96])

Decoder shapes:
d1 after ConvTranspose: torch.Size([4, 256, 256, 192])
d1 after upsample: torch.Size([4, 256, 128, 96])
e3 for concatenation: torch.Size([4, 256, 128, 96])
d1_cat: torch.Size([4, 512, 128, 96])
d2 before upsample: torch.Size([4, 128, 128, 96])
d2 after upsample: torch.Size([4, 128, 256, 192])
e2 for concatenation: torch.Size([4, 128, 256, 192])
d2_cat: torch.Size([4, 256, 256, 192])
d3 before upsample: torch.Size([4, 128, 256, 192])
d3 after upsample: torch.Size([4, 128, 512, 384])
e1 for concatenation: torch.Size([4, 64, 512, 384])
d3_cat: torch.Size([4, 192, 512, 384])

Final output: torch.Size([4, 3, 512, 384])
